In [ ]:
import h5py
import pandas as pd
import numpy as np
import json
import os
from os.path import join
import tempfile
import negspy.coordinates as nc

In [ ]:
f = h5py.File("../data_processing_20200429/my_file_genome_wide_20180228.multires.mv5", "r")

In [ ]:
def descend_obj(obj,sep='\t'):
    """
    Iterate through groups in a HDF5 file and prints the groups and datasets names and datasets attributes
    """
    if type(obj) in [h5py._hl.group.Group,h5py._hl.files.File]:
        for key in obj.keys():
            print(sep,'-',key,':',obj[key])
            descend_obj(obj[key],sep=sep+'\t')
    elif type(obj)==h5py._hl.dataset.Dataset:
        for key in obj.attrs.keys():
            print(sep+'\t','-',key,':',obj.attrs[key])

In [ ]:
descend_obj(f)

In [ ]:
type(f)

In [ ]:
def create_multivec_file():
    tf = tempfile.TemporaryFile()
    f = h5py.File(tf, 'w')
    
    # Create level zero groups
    info_group = f.create_group("info")
    resolutions_group = f.create_group("resolutions")
    
    # Prepare to fill in chroms dataset
    chromosomes = nc.get_chromorder('hg38')
    num_chromosomes = len(chromosomes)
    chroms_length_arr = np.array([ nc.get_chrominfo('hg38').chrom_lengths[x] for x in chromosomes ], dtype="i8")
    chroms_name_arr = np.array(chromosomes, dtype="S23")
    
    def create_chroms_group(d):
        chroms_group = d.create_group("chroms")
        # Fill in chroms dataset entries "length" and "name"
        chroms_group.create_dataset("length", data=chroms_length_arr)
        chroms_group.create_dataset("name", data=chroms_name_arr)
    
    create_chroms_group(f)
    
    # Prepare to fill in resolutions dataset
    resolutions = [ str(1000*(2**x)) for x in range(15)]
    print(resolutions)
    
    # Fill in resolutions dataset entries
    for resolution in resolutions:
        # Create each resolution group
        resolution_group = resolutions_group.create_group(resolution)
        
        create_chroms_group(resolution_group)
        resolution_values_group = resolution_group.create_group("values")
        
        for chromosome in chromosomes:
            # TODO: fill in the chromosome values
            chrom_dataset = resolution_group.create_dataset(chromosome, (1, 1), dtype="f4")
        
    return f
    

In [ ]:
len(nc.get_chromorder('hg38'))

In [ ]:
nc.get_chrominfo('hg38').chrom_lengths["chr2"]

In [ ]:
descend_obj(create_multivec_file())